In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

In [2]:
examples = [
    {
        "movie_name": "탑건",
        "answer": "🛩️✈️🔥",
    },
    {
        "movie_name": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "영화 제목에 대하여 이모티콘 3개로만 표현해줘"), ("ai", "{answer}")]
)

example_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [3]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

helpful_ai_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 세계의 영화 정보를 모두 알고 있어. 내가 영화 제목을 말하면 예제처럼 이모티콘으로 제목을 표현해줘"),
        example_template,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


movie_emoticon_chain = RunnablePassthrough.assign(history=load_memory) | helpful_ai_prompt | llm


def invoke_chain(question):
    result = movie_emoticon_chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)


invoke_chain("모던타임즈 이모티콘")


content='⏰🕰️🎥'


In [4]:
invoke_chain("다크나이트 이모티콘")

content='🦇🃏🌃'


In [5]:
invoke_chain("내가 첫번째 질문한 영화 알려줘")

content='첫번째 질문한 영화는 "탑건"입니다.'


In [6]:
invoke_chain("그건 예제로 넣어 준 자료잖아, 예제를 제외하고 말해줘")

content='알겠어, 죄송해요. 첫번째 질문한 영화는 "모던 타임즈"입니다.'


In [7]:
invoke_chain("내가 첫번째 질문한 영화 알려줘")

content='첫번째 질문한 영화는 "Top Gun"이었습니다.'


In [8]:
invoke_chain("내가 네번째 질문한 영화 알려줘")

content='네번째로 질문한 영화는 "The Dark Knight"입니다.'
